<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/main/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from bs4 import BeautifulSoup as b4
import asyncio
import multiprocessing
import time
import requests
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


### Download every chapter

In [12]:
def getLinks(url):
  driver = webdriver.Chrome()
  driver.get(url)
  #wait for the page to load
  #start a driver for chrome
  #see if I can use the adblocker extension
  #go to the link through the driver
  #download the webpage? Or see if I can parse it using selenium/beautifulsoup
  #get all the links in the page with beautifulsoup
  #multithreaded download all the links
  #when the links are exhausted, go to the next tab, if avaliable
  #continue until at the final chapter
  #

In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver

url = "http://legendas.tv/busca/walking%20dead%20s03e02"
browser = webdriver.PhantomJS()
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'lxml')
a = soup.find('section', 'wrapper')

AttributeError: module 'selenium.webdriver' has no attribute 'PhantomJS'

In [30]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

options = webdriver.FirefoxOptions()
options.add_argument('--headless')
browser = webdriver.Firefox(options=options)
#executable_path='YOUR_PATH/geckodriver.exe'
try:
    browser.get("https://www.lightnovelpub.com/novel/chrysalis-552/chapter-1-30041322")
    timeout_in_seconds = 10
    WebDriverWait(browser, timeout_in_seconds).until(ec.presence_of_element_located((By.ID, 'chapter-container')))
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    html_content = str(soup)

    # Define the filename
    filename = "output.html"

    # Write the HTML content to the file
    with open(filename, "w") as file:
        file.write(html_content)
    file.close()
    #print(soup.get_text())
    #soup.prettify()
    #print(soup.find_all('p'))
except TimeoutException:
    print("I give up...")
finally:
    browser.quit()

In [66]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

with open("output.html", "r") as file:
  soup = BeautifulSoup(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  for t in tag:
    if "Chapter" in t.get_text():
      print(t.get_text())
  soup = soup.find_all('p')
  for p in soup:
    print(p.get_text())
  file.close()

Chapter 1: Anthony Reborn
I had no time to think, pause or consider before I came to be enveloped in a soft, white light. A blinding flash impacting not just my eyes but seemingly my whole body.
Sensation fell gradually away as I tried to gather my thoughts. What had happened? Where was I? I think I made myself dinner, my parents hadn’t been home, again. I was eating in my room, at my desk maybe? Then there had been ... white?
I can’t tell how long I’ve been here, floating in this gentle light. Has it been ten days? Or only ten minutes? The light surrounds me, enveloping me like a warm blanket.
Sound begins to swell like a wave in the distance. It rushes toward me and just when washes over me I hear a voice.
[Welcome Anthony]
Well thank you! These are the kinds of good manners that make me feel warm inside.
[you have died]
Dammit!
What do you mean I died, disembodied voice? How on earth are you speaking anyway? Come to think of it, how on earth am I hearing you? Do I even have ears rig

In [48]:
soup = BeautifulSoup(html, 'html.parser')


None


### Cull each URL

In [ ]:
def cullURL(url):
  #cull scripts
  #edit the title
  #edit the header
  #

### Input url

In [13]:
url = input("Enter URL: ")
#sample url: https://www.lightnovelpub.com/novel/chrysalis-16091353/chapters
getLinks(url)



Enter URL: https://www.lightnovelpub.com/novel/chrysalis-552/chapter-1-30041322


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/124.0.6367.91/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x587c9c522cb3 <unknown>
#1 0x587c9c2114a7 <unknown>
#2 0x587c9c245a26 <unknown>
#3 0x587c9c241980 <unknown>
#4 0x587c9c28a7b0 <unknown>
#5 0x587c9c27e1b3 <unknown>
#6 0x587c9c24f24a <unknown>
#7 0x587c9c24fc1e <unknown>
#8 0x587c9c4e6fdb <unknown>
#9 0x587c9c4eaf2b <unknown>
#10 0x587c9c4d30f1 <unknown>
#11 0x587c9c4eba92 <unknown>
#12 0x587c9c4b7faf <unknown>
#13 0x587c9c511e08 <unknown>
#14 0x587c9c511fe0 <unknown>
#15 0x587c9c521e04 <unknown>
#16 0x792c51cf9ac3 <unknown>
